In [1]:
import scipy.sparse
import matplotlib
from functools import partial
from scipy import interpolate
from itertools import cycle
import pandas as pd
import pyro
import torch
import anndata
import scanpy as sc 
import numpy as np
import sys
import matplotlib.pyplot as plt 

sys.path.insert(1, '/nfs/team205/vk7/sanger_projects/BayraktarLab/scvi-tools/')

import scvi


%config InlineBackend.figure_format='retina'

In [2]:
NOTEBOOK_NUMBER = 1

torch.cuda.set_device(f'cuda:{NOTEBOOK_NUMBER}')

table_dir = '/nfs/team205/vk7/sanger_projects/cell2location_paper/notebooks/selected_results/benchmarking/with_tissue_zones/real_mg/pyro/hyperparameters/'

sc_data_folder = '/nfs/team205/vk7/sanger_projects/cell2location_paper/notebooks/selected_data/mouse_visium_snrna/'
sp_data_folder = '/nfs/team205/vk7/sanger_projects/cell2location_paper/notebooks/selected_results/benchmarking/with_tissue_zones/data/'
sc_results_folder = '/nfs/team205/vk7/sanger_projects/cell2location_paper/notebooks/selected_results/benchmarking/with_tissue_zones/real_mg/'
results_folder = '/nfs/team205/vk7/sanger_projects/cell2location_paper/notebooks/selected_results/benchmarking/with_tissue_zones/real_mg/pyro/'
# results_folder = table_dir

scvi_run_name = f'{results_folder}test_regression_normalisation_adam_lr0002_test_data'

In [3]:
conditions = pd.read_csv(f'{table_dir}/param_tables/table_{NOTEBOOK_NUMBER}.csv')

In [4]:
adata = anndata.read(f'{sp_data_folder}synth_adata_real_mg_20210131.h5ad')
adata_vis = adata[adata.obs['sample']=='exper0']
adata_vis.uns['spatial'] = {'x': 'y'}

/nfs/team283/vk7/software/miniconda3farm5/envs/scvi-env2/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
Trying to set attribute `.uns` of view, copying.


In [5]:
# Save anndata object with signatures
adata_file = f"{scvi_run_name}_ref/sc.h5ad"
adata_snrna_raw = sc.read_h5ad(adata_file)

# export estimated expression in each cluster
if 'means_per_cluster_mu_fg' in adata_snrna_raw.varm.keys():
    regr_inf_aver = adata_snrna_raw.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}' 
                                        for i in adata_snrna_raw.uns['mod']['factor_names']]].copy()
else:
    regr_inf_aver = adata_snrna_raw.var[[f'means_per_cluster_mu_fg_{i}' 
                                        for i in adata_snrna_raw.uns['mod']['factor_names']]].copy()
regr_inf_aver.columns = adata_snrna_raw.uns['mod']['factor_names']
regr_inf_aver.iloc[0:5, 0:5]

,Astro_AMY,Astro_AMY_CTX,Astro_CTX,Astro_HPC,Astro_HYPO
ENSEMBL,,,,,
ENSMUSG00000051951,0.050511,0.105329,0.039299,0.176706,0.032688
ENSMUSG00000089699,0.031588,0.025723,0.008132,0.014632,0.032263
ENSMUSG00000033845,0.140224,0.064763,0.073241,0.065900,0.087580
ENSMUSG00000033813,0.085916,0.130148,0.180947,0.105858,0.070225
ENSMUSG00000002459,2.846514,3.509084,3.756964,4.002057,1.390353


In [6]:
# rename genes to ENSEMBL
adata_vis.var['SYMBOL'] = adata_vis.var_names

# find shared genes and subset both anndata and reference signatures
intersect = np.intersect1d(adata_vis.var_names, regr_inf_aver.index)
adata_vis = adata_vis[:, intersect].copy()
regr_inf_aver = regr_inf_aver.loc[intersect, :].copy()

# prepare anndata for scVI model
scvi.data.setup_anndata(adata=adata_vis, batch_key="sample")
scvi.data.view_anndata_setup(adata_vis)

/nfs/team283/vk7/software/miniconda3farm5/envs/scvi-env2/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


INFO     Using batches from adata.obs["sample"]                                              
INFO     No label_key inputted, assuming all cells have same label                           
INFO     Using data from adata.X                                                             
INFO     Computing library size prior per batch                                              
INFO     Successfully registered anndata object containing 2500 cells, 12080 vars, 1 batches,
         1 labels, and 0 proteins. Also registered 0 extra categorical covariates and 0 extra
         continuous covariates.                                                              
INFO     Please do not further modify adata until model is trained.                          


/nfs/team283/vk7/software/miniconda3farm5/envs/scvi-env2/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [7]:
from scvi.external.cell2location._cell2location_v3_module import LocationModelLinearDependentWMultiExperimentLocationBackgroundNormLevelGeneAlphaPyroModel

In [8]:
!pwd

/nfs/team205/vk7/sanger_projects/cell2location_paper/notebooks/selected_results/benchmarking/with_tissue_zones/real_mg/pyro/hyperparameters/param_tables


In [ ]:
pyro.clear_param_store()

history = []

for _, con in conditions.iterrows():
    
    ########### REGRESSION MODEL #############
    
    ########### CELL2LOCATION MODEL #############
    mod = scvi.external.Cell2location(
        adata_vis, cell_state_df=regr_inf_aver.astype('float32'), 
        model_class=LocationModelLinearDependentWMultiExperimentLocationBackgroundNormLevelGeneAlphaPyroModel,
        amortised=False, #encoder_mode="single-multiple",
       # encoder_kwargs={'dropout_rate': 0.2},
        # the expected average cell abundance - user-provided, tissue-dependent hyper-prior:
        N_cells_per_location=con["N_cells_per_location"],
        A_factors_per_location=con["A_factors_per_location"],
        B_groups_per_location=con["B_groups_per_location"],
        detection_alpha=con["detection_alpha"],
    ) 
    mod.train(max_epochs=con['max_epochs'], 
              batch_size=None, 
              train_size=1,
              #plan_kwargs={
              #    'optim': pyro.optim.ClippedAdam({
              #        'lr': con['lr'], 
              #        'clip_norm': con['clip_norm']
              #    })
              #},
              plan_kwargs={'optim': pyro.optim.Adam({'lr': con['lr']})},
              use_gpu=True
    )
    #history += list(mod.history['elbo_train'].values[:, 0])

    adata_vis_res = mod.export_posterior(
        adata_vis, sample_kwargs={'num_samples': 1000, 'batch_size': mod.adata.n_obs, 'use_gpu': True})

    scvi_run_name = f'{table_dir}/{con["names"]}'

    mod.save(f"{scvi_run_name}", overwrite=True)
    adata_file = f"{scvi_run_name}/sp.h5ad"
    adata_vis_res.write(adata_file)

    # plot ELBO loss history during training, removing first 100 epochs from the plot
    mod.plot_history(5000)
    plt.savefig(f"{scvi_run_name}/training_ELBO_history_minus5k.png",
                bbox_inches='tight')
    plt.close()
    mod.plot_history(0)
    plt.savefig(f"{scvi_run_name}/training_ELBO_history_all.png",
                bbox_inches='tight')
    plt.close()
       
    # Examine reconstruction accuracy to assess if there are any issues with mapping
    # the plot should be roughly diagonal, strong deviations will signal problems
    mod.plot_QC()
    plt.savefig(f"{scvi_run_name}/reconstruction_accuracy_histogram.png",
                bbox_inches='tight')
    plt.close()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
/nfs/team283/vk7/software/miniconda3farm5/envs/scvi-env2/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: you passed in a val_dataloader but have no validation_step. Skipping val loop
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5]


Sampling global variables, sample: 100%|██████████| 999/999 [00:21<00:00, 46.28it/s]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
/nfs/team283/vk7/software/miniconda3farm5/envs/scvi-env2/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: you passed in a val_dataloader but have no validation_step. Skipping val loop
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5]


Sampling global variables, sample: 100%|██████████| 999/999 [00:51<00:00, 19.44it/s]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
/nfs/team283/vk7/software/miniconda3farm5/envs/scvi-env2/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: you passed in a val_dataloader but have no validation_step. Skipping val loop
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5]


Epoch 4983/40000:  12%|█▏        | 4982/40000 [14:22<1:39:11,  5.88it/s, v_num=1, elbo_train=3.2e+7] 